In [ ]:
# 🌍 Simple Transformer Attention Visualizer (English → German)
# =============================================================
# This code:
# 1️⃣ Translates English → German using a small pre-trained model (MarianMT)
# 2️⃣ Visualizes how German words attend to English words (cross-attention)
# -------------------------------------------------------------



# ✅ Lightweight Translation with Attention Visualization (T5-small)
# Works on CPU and Visual Studio, uses < 2 GB RAM

%pip install transformers torch sentencepiece matplotlib seaborn --quiet

from transformers import T5ForConditionalGeneration, T5Tokenizer
import torch, matplotlib.pyplot as plt, seaborn as sns

# -----------------------------
# 1️⃣ Load lightweight model
# -----------------------------
model_name = "google-t5/t5-small"   # ✅ very small and Hugging Face verified
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name, output_attentions=True)
model.eval();

# -----------------------------
# 2️⃣ Input sentence
# -----------------------------
sentence = "translate English to German: I love AI."

# Tokenize
inputs = tokenizer(sentence, return_tensors="pt")

# -----------------------------
# 3️⃣ Generate translation
# -----------------------------
with torch.no_grad():
    output = model.generate(
        **inputs,
        output_attentions=True,
        return_dict_in_generate=True,
        max_length=40
    )

# Decode translation
translation = tokenizer.decode(output.sequences[0], skip_special_tokens=True)
print(f"English: I love AI.")
print(f"German : {translation}")

# -----------------------------
# 4️⃣ Visualize encoder self-attention
# -----------------------------
# Get attention from the encoder (last layer, first head)
enc_attn = model(**inputs, output_attentions=True).encoder_attentions[-1][0,0].cpu().numpy()

src_tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])

plt.figure(figsize=(6,5))
sns.heatmap(enc_attn, xticklabels=src_tokens, yticklabels=src_tokens, cmap="YlGnBu")
plt.title("Encoder Self-Attention (English Tokens)")
plt.xlabel("Keys")
plt.ylabel("Queries")
plt.tight_layout()
plt.show()




[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


/Users/swatichandna/.pyenv/versions/3.11.7/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


: 